<a href="https://colab.research.google.com/github/Calcifer777/learn-nlp/blob/main/learn-transformers/ner_ft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [16]:
import pandas as pd

import transformers
from datasets import load_dataset

In [ ]:
ds = load_dataset("xtreme", name="PAN-X.it")

In [18]:
ds.set_format("pd")

In [19]:
pd.DataFrame(ds["train"][:5])

,tokens,ner_tags,langs
0,"[', '', Andrew, Noble, '', ', -, fisico, brita...","[0, 0, 1, 2, 0, 0, 0, 0, 5]","[it, it, it, it, it, it, it, it, it]"
1,"[Eliminato, al, 4T, da, Andy, Murray, [3]]","[0, 0, 0, 0, 1, 2, 0]","[it, it, it, it, it, it, it]"
2,"[', '', Suzuki, '', ']","[0, 0, 3, 0, 0]","[it, it, it, it, it]"
3,"[Seekirchen, am, Wallersee]","[5, 6, 6]","[it, it, it]"
4,"[Ha, mutuato, il, nome, dal, capoluogo, Tarfay...","[0, 0, 0, 0, 0, 0, 5, 0]","[it, it, it, it, it, it, it, it]"


In [20]:
ds.reset_format()

In [21]:
class_labels = ds["train"].features["ner_tags"].feature

In [23]:
ds = ds.map(
    function=lambda batch: {"ner_tags_str": [class_labels.int2str(tag) for tag in batch["ner_tags"]]},
    batched=True,
    batch_size=64,
)

  0%|          | 0/313 [00:00<?, ?ba/s]

  0%|          | 0/157 [00:00<?, ?ba/s]

  0%|          | 0/157 [00:00<?, ?ba/s]

In [43]:
tags_freqs = dict()

for split, ds_split in ds.items():
  df_split = pd.DataFrame(ds_split[:])
  tmp = df_split.explode("ner_tags_str").groupby("ner_tags_str").size()
  tags_freqs[split] = tmp / tmp.sum()

pd.DataFrame.from_dict(tags_freqs, orient="columns")

,train,validation,test
ner_tags_str,,,
B-LOC,0.060633,0.057496,0.059601
B-ORG,0.049512,0.051447,0.050537
B-PER,0.060103,0.061384,0.059873
I-LOC,0.054164,0.052072,0.053987
I-ORG,0.107842,0.112393,0.114033
I-PER,0.072203,0.073283,0.071645
O,0.595543,0.591926,0.590323
